You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan Document

Title: The Future of Artificial Intelligence: Latest Trends and Key Players

Outline:

I. Introduction
- Definition of Artificial Intelligence (AI)
- Importance and impact of AI in various industries
- Brief overview of the latest trends and key players in the AI industry

II. Latest Trends in Artificial Intelligence
A. Machine Learning
- Overview of machine learning algorithms and applications
- Latest advancements in machine learning technology
- Case studies of successful machine le

I now can give a great answer

Final Answer:

# The Future of Artificial Intelligence: Latest Trends and Key Players

## Introduction
Artificial Intelligence (AI) has become a driving force in transforming various industries, revolutionizing the way we work and interact with technology. Defined as the simulation of human intelligence processes by machines, AI has permeated our daily lives, from personalized recommendations on streaming services to autonomous vehicles on our roads. The importance and impact of AI cannot be understated, with its ability to streamline processes, improve decision-making, and enhance overall efficiency. In this blog post, we will delve into the latest trends and key players shaping the AI landscape today.

## Latest Trends in Artificial Intelligence

### Machine Learning
Machine Learning, a subset of AI, focuses on the development of algorithms that enable machines to learn from and make predictions based on data. Recent advancements in machine learning tec

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Latest Trends and Key Players

## Introduction
Artificial Intelligence (AI) has become a driving force in transforming various industries, revolutionizing the way we work and interact with technology. Defined as the simulation of human intelligence processes by machines, AI has permeated our daily lives, from personalized recommendations on streaming services to autonomous vehicles on our roads. The importance and impact of AI cannot be understated, with its ability to streamline processes, improve decision-making, and enhance overall efficiency. In this blog post, we will delve into the latest trends and key players shaping the AI landscape today.

## Latest Trends in Artificial Intelligence

### Machine Learning
Machine Learning, a subset of AI, focuses on the development of algorithms that enable machines to learn from and make predictions based on data. Recent advancements in machine learning technology have propelled the field forward, with breakthroughs in deep learning, reinforcement learning, and neural networks. Organizations across various sectors are leveraging machine learning to optimize operations, personalize user experiences, and drive innovation. Case studies showcasing successful machine learning projects highlight the transformative power of this technology in areas such as healthcare, finance, and marketing.

### Natural Language Processing (NLP)
Natural Language Processing (NLP) enables machines to understand, interpret, and generate human language. Recent developments in NLP technology, fueled by advancements in deep learning and neural network models, have enhanced the capabilities of AI-powered language processing systems. Use cases of NLP in real-world applications include sentiment analysis, language translation, and chatbot interactions, showcasing the versatility and potential impact of this technology in enhancing communication and user experiences.

### Computer Vision
Computer Vision focuses on enabling machines to interpret and analyze visual information from the world around them. Recent innovations in computer vision systems, driven by advancements in image recognition, object detection, and video analysis, have revolutionized industries such as healthcare, retail, and security. Examples of computer vision technology in action include facial recognition systems, autonomous drones, and augmented reality applications, demonstrating the vast potential of visual data processing in various domains.

## Key Players in the AI Industry

### Tech Giants
Leading tech companies such as Google, Amazon, Microsoft, and IBM are at the forefront of AI innovation, investing heavily in research and development to push the boundaries of artificial intelligence. These tech giants have developed sophisticated AI solutions, from virtual assistants to autonomous vehicles, and continue to shape the industry with their AI strategies and initiatives. Their impact on the development of AI technology is profound, influencing trends, setting standards, and driving the adoption of AI across sectors.

### Startups and Innovators
Emerging AI startups are making waves in the industry, bringing fresh perspectives and innovative solutions to the table. These startups, with their agility and creativity, are disrupting traditional approaches to AI and driving rapid advancements in the field. Their unique contributions to AI technology, whether in healthcare, finance, or cybersecurity, have the potential to reshape industries and challenge established players in the market. By fostering a culture of innovation and experimentation, these startups are paving the way for the future of artificial intelligence.

## Conclusion
As artificial intelligence continues to evolve and expand its reach, staying informed about the latest trends and key players in the industry is essential for tech enthusiasts, business leaders, students, and anyone curious about the future of AI. By exploring the latest advancements in machine learning, NLP, and computer vision, and highlighting the contributions of tech giants and startups in shaping the AI landscape, we gain valuable insights into the transformative power of AI technology. The future of artificial intelligence is bright, promising new possibilities and opportunities for innovation and growth.

*Stay informed about the latest trends and developments in artificial intelligence by subscribing to our newsletter for regular updates and insights.*

This blog post provides a comprehensive overview of the latest trends and key players in the field of artificial intelligence, offering valuable insights and perspectives for readers interested in AI technology. By incorporating relevant SEO keywords and credible resources, the post aims to engage a diverse audience and provide a deeper understanding of the dynamic and evolving AI landscape.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [17]:
topic = "Nvidia"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Nvidia.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan Document:

Title: Unlocking the Power of Nvidia: Latest Trends, Key Players, and Noteworthy News

Outline:
I. Introduction
- Brief overview of Nvidia as a leading technology company in the GPU industry
- Importance of staying updated with the latest trends and news in the Nvidia ecosystem

II. Latest Trends in Nvidia
- AI and Machine Learning advancements
- Gaming industry innovations
- Data center solutions for cloud computing

III. Key Players in the Nvidia Ecosystem
- Nvidia's leadership team and key executiv

I now can give a great answer

Final Answer:

# Unlocking the Power of Nvidia: Latest Trends, Key Players, and Noteworthy News

Nvidia stands as a prominent figure in the technology industry, specifically known for its contributions to the GPU sector. With a focus on innovation and pushing the boundaries of graphics technology, Nvidia has established itself as a leader in the field. Staying informed about the latest trends and news surrounding Nvidia is crucial for tech enthusiasts, gamers, and businesses alike to understand the evolving landscape of GPU technology.

Nvidia's advancements in AI and Machine Learning have been groundbreaking, with the company consistently pushing the boundaries of what is possible in these fields. From autonomous vehicles to healthcare applications, Nvidia's technology plays a vital role in driving innovation. Additionally, Nvidia's impact on the gaming industry cannot be understated, with their graphics cards revolutionizing the gaming experience for pl

In [19]:
Markdown(result)

# Unlocking the Power of Nvidia: Latest Trends, Key Players, and Noteworthy News

Nvidia stands as a prominent figure in the technology industry, specifically known for its contributions to the GPU sector. With a focus on innovation and pushing the boundaries of graphics technology, Nvidia has established itself as a leader in the field. Staying informed about the latest trends and news surrounding Nvidia is crucial for tech enthusiasts, gamers, and businesses alike to understand the evolving landscape of GPU technology.

Nvidia's advancements in AI and Machine Learning have been groundbreaking, with the company consistently pushing the boundaries of what is possible in these fields. From autonomous vehicles to healthcare applications, Nvidia's technology plays a vital role in driving innovation. Additionally, Nvidia's impact on the gaming industry cannot be understated, with their graphics cards revolutionizing the gaming experience for players around the world. Furthermore, Nvidia's data center solutions for cloud computing have paved the way for more efficient and powerful computing capabilities.

Nvidia's leadership team, including key executives such as CEO Jensen Huang, plays a pivotal role in driving the company's vision and strategy forward. Collaborations and partnerships with other technology companies further enhance Nvidia's reach and influence in the industry. Moreover, Nvidia has cultivated a vibrant community of developers and enthusiasts who contribute to the growth and development of the Nvidia ecosystem.

Keeping up with Nvidia's recent product launches and updates is essential to understanding the company's direction and impact on the market. Monitoring Nvidia's financial performance and market trends provides valuable insights into the company's growth and success. Additionally, industry accolades and recognition highlight Nvidia's position as a leading innovator in the GPU industry.

In conclusion, Nvidia's latest trends, key players, and noteworthy news are essential components for tech enthusiasts, gamers, and businesses to stay informed about the evolving landscape of GPU technology. By following Nvidia's official channels and industry news sources, individuals can stay ahead of the curve and gain valuable insights into Nvidia's developments. As Nvidia continues to push the boundaries of graphics technology, staying informed and engaged with the company's latest updates is crucial for anyone interested in the future of GPU technology.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).